In [1]:
import pandas as pd
import numpy as np
from config_loader import load
import argparse
import sys
import seaborn as sns
from MyDataUnderstanding import featureAnalysis
from MyPreprocessing import MyPreprocessing
import numpy as np
from time import time
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [3]:
def getData(path, filenames_type):
    '''
    features_lst = [
        "Pclass", "Survived", "Name", "Sex", "Age",
        "Sibsp", "Parch", "Ticket", "Fare", "Cabin","Embarked",
        "Boat","Body","Home.dest"]
    '''
    if filenames_type == 'train':
        filename = 'train'
    elif filenames_type == 'test':
        filename = 'test'
    else:
        filename = 'titanicAll'
        
    df_features = pd.read_csv(path + filename + '.csv',
                           sep=',')

    if filename not in ['train', 'test']:
        # drop unnecessary columns that don't exist in the official data
        df_features.drop(['Boat', 'Body', 'Home.dest'],
                           axis=1,
                           inplace=True)
    #labels = df_features['Survived']
    #df_features = df_features.drop(['Survived'], axis=1)
    return df_features

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "-c", "--config", default="titanic.cfg",
    help="specify the location of the clustering config file"
)
args, _ = parser.parse_known_args()

config_file = args.config
config = load(config_file)

##
verbose = config.get('titanic', 'verbose')
path = config.get('titanic', 'path') + '/'
file_type = config.get('titanic', 'file_type')

filename_type = 'train'
if file_type == 'all':
    filename_type = 'other'

data = getData(path, filename_type)

In [5]:
featureAnalysis(data)

Data summary
            Pclass     Survived          Age        SibSp        Parch         Fare
count  1309.000000  1309.000000  1046.000000  1309.000000  1309.000000  1308.000000
mean      2.294882     0.381971    29.881138     0.498854     0.385027    33.295479
std       0.837836     0.486055    14.413493     1.041658     0.865560    51.758668
min       1.000000     0.000000     0.170000     0.000000     0.000000     0.000000
25%       2.000000     0.000000    21.000000     0.000000     0.000000     7.895800
50%       3.000000     0.000000    28.000000     0.000000     0.000000    14.454200
75%       3.000000     1.000000    39.000000     1.000000     0.000000    31.275000
max       3.000000     1.000000    80.000000     8.000000     9.000000   512.329200

Categorical data summary
                    Name   Sex    Ticket        Cabin Embarked
count               1309  1309      1309          295     1307
unique              1307     2       929          186        3
top     Kelly, M

* Names are unique across the dataset
* Sex variable as two possible values with 63% male (top=male, freq=843/count(=1309)).
* Cabin values have several dupicates across samples or several passengers shared a cabin but there are also many missing values (77%)
* Embarked takes three possible values. S port used by most passengers (top=S)
* Ticket feature has high ratio (29%) of duplicate values

In [6]:
## Preprocessing
preprocess = MyPreprocessing()
preprocess.fit(data)
df = preprocess.new_df
# the labels "Survived"
labels = preprocess.labels_
print(labels.head())
# the initial dataset without any preprocessing
print(preprocess.df_initial.head())
# the preprocessed data
print(preprocess.new_df.head())

0    1
1    1
2    0
3    0
4    0
Name: Survived, dtype: int64
   Pclass                                             Name     Sex    Age  SibSp  Parch  Ticket      Fare    Cabin Embarked   Title
0       1                    Allen, Miss. Elisabeth Walton  female  29.00      0      0   24160  211.3375       B5        S    Miss
1       1                   Allison, Master. Hudson Trevor    male   0.92      1      2  113781  151.5500  C22 C26        S  Master
2       1                     Allison, Miss. Helen Loraine  female   2.00      1      2  113781  151.5500  C22 C26        S    Miss
3       1             Allison, Mr. Hudson Joshua Creighton    male  30.00      1      2  113781  151.5500  C22 C26        S      Mr
4       1  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female  25.00      1      2  113781  151.5500  C22 C26        S     Mrs
        Age  SibSp     Parch      Fare  Pclass_2  Pclass_3  Sex_male  Embarked_Q  Embarked_S  Title_Mr0  Title_Mr50  Title_Mrs  Title_Ms  Title_